In [1]:
from pathlib import Path
import os

# 1. 작업 경로 설정 (pathlib 사용)
base_path = Path("C:/Users/it/Documents/Python-workspace/Ai-ML-DL-workspace-0930/memo/3-3-Yolov8")

# 경로가 존재하지 않으면 생성
base_path.mkdir(parents=True, exist_ok=True)
print(f"기본 작업 경로가 설정되었습니다: {base_path}")

기본 작업 경로가 설정되었습니다: C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-3-Yolov8


In [2]:
# 2. GPU 확인 (NVIDIA GPU가 없는 경우 오류가 출력되지만 정상입니다)
!nvidia-smi

'nvidia-smi'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
# 3. 필수 라이브러리 설치
!pip install ultralytics requests pyyaml pathlib

In [4]:
# 4. Ultralytics 환경 확인
from IPython.display import display, Image
from IPython import display
display.clear_output()
!yolo checks



Ultralytics 8.3.214  Python-3.10.18 torch-2.8.0+cpu CPU (Intel Core i5-10400 2.90GHz)
Setup complete  (12 CPUs, 15.7 GB RAM, 193.5/232.2 GB disk)

OS                     Windows-10-10.0.19045-SP0
Environment            Windows
Python                 3.10.18
Install                pip
Path                   C:\Users\it\anaconda3\envs\pytorch\Lib\site-packages\ultralytics
RAM                    15.65 GB
Disk                   193.5/232.2 GB
CPU                    Intel Core i5-10400 2.90GHz
CPU count              12
GPU                    None
GPU count              None
CUDA                   None

numpy                   2.0.1>=1.23.0
matplotlib              3.10.6>=3.3.0
opencv-python           4.12.0.88>=4.6.0
pillow                  11.1.0>=7.1.2
pyyaml                  6.0.2>=5.3.1
requests                2.32.5>=2.23.0
scipy                   1.15.3>=1.4.1
torch                   2.8.0>=1.8.0
torch                   2.8.0!=2.4.0,>=1.8.0; sys_platform == "win32"
torchvision      

In [5]:
import requests
from pathlib import Path

# 5. 데이터셋 다운로드
url = "https://public.roboflow.com/ds/csaXb5Dicq?key=6xq5P1FIPU"
zip_path = base_path / "Aquarium_DataSet.zip"

if not zip_path.exists():
    print(f"데이터셋을 다운로드합니다...URL: {url}저장 위치: {zip_path}")
    try:
        response = requests.get(url, stream=True, timeout=30)
        response.raise_for_status()
        with open(zip_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("데이터셋 다운로드 완료.")
    except requests.exceptions.RequestException as e:
        print(f"데이터셋 다운로드 중 오류 발생: {e}")
else:
    print(f"데이터셋이 이미 존재합니다: {zip_path}")

데이터셋이 이미 존재합니다: C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-3-Yolov8\Aquarium_DataSet.zip


In [6]:
import zipfile
from pathlib import Path

# 6. 데이터셋 압축 해제
zip_path = base_path / "Aquarium_DataSet.zip"
extract_path = base_path / "Aquarium_DataSet"

extract_path.mkdir(parents=True, exist_ok=True)

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"데이터셋 압축 해제가 완료되었습니다. 폴더 경로: {extract_path}")
except FileNotFoundError:
    print(f"오류: 압축할 파일을 찾을 수 없습니다 - {zip_path}")
    print("이전 5번 셀에서 데이터셋이 정상적으로 다운로드되었는지 확인하세요.")
except zipfile.BadZipFile:
    print(f"오류: ZIP 파일이 손상되었거나 올바른 ZIP 파일이 아닙니다. - {zip_path}")
    print("파일을 삭제하고 5번 셀부터 다시 실행하여 다운로드해 보세요.")

데이터셋 압축 해제가 완료되었습니다. 폴더 경로: C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-3-Yolov8\Aquarium_DataSet


In [7]:
from pathlib import Path
import yaml

# 7. data.yaml 파일 경로 수정 및 새 파일 생성
original_yaml_path = extract_path / "data.yaml"
new_yaml_path = base_path / "Aquarium_DataSet_Corrected.yaml"

if original_yaml_path.exists():
    with open(original_yaml_path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
        print("--- 원본 data.yaml 내용 ---")
        print(data)

    # pathlib을 사용하여 경로를 절대 경로로, 슬래시로 통일하여 수정
    data['train'] = str(extract_path / 'train' / 'images')
    data['val'] = str(extract_path / 'valid' / 'images')
    data['test'] = str(extract_path / 'test' / 'images')

    # 수정된 내용으로 새 YAML 파일 작성
    with open(new_yaml_path, 'w', encoding='utf-8') as f:
        yaml.dump(data, f, allow_unicode=True, sort_keys=False)

    print("--- 경로가 수정된 data.yaml 내용 ---")
    print(data)
    print(f"수정된 YAML 파일이 다음 경로에 저장되었습니다: {new_yaml_path}")
else:
    print(f"원본 data.yaml 파일을 찾을 수 없어 경로 수정에 실패했습니다: {original_yaml_path}")

--- 원본 data.yaml 내용 ---
{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 7, 'names': ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray'], 'roboflow': {'workspace': 'brad-dwyer', 'project': 'aquarium-combined', 'version': 2, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/brad-dwyer/aquarium-combined/dataset/2'}}
--- 경로가 수정된 data.yaml 내용 ---
{'train': 'C:\\Users\\it\\Documents\\Python-workspace\\Ai-ML-DL-workspace-0930\\memo\\3-3-Yolov8\\Aquarium_DataSet\\train\\images', 'val': 'C:\\Users\\it\\Documents\\Python-workspace\\Ai-ML-DL-workspace-0930\\memo\\3-3-Yolov8\\Aquarium_DataSet\\valid\\images', 'test': 'C:\\Users\\it\\Documents\\Python-workspace\\Ai-ML-DL-workspace-0930\\memo\\3-3-Yolov8\\Aquarium_DataSet\\test\\images', 'nc': 7, 'names': ['fish', 'jellyfish', 'penguin', 'puffin', 'shark', 'starfish', 'stingray'], 'roboflow': {'workspace': 'brad-dwyer', 'project': 'aquarium-combined', 'version': 2, 'license':

In [8]:
import requests
from pathlib import Path

# 8. 사전 학습된 YOLOv8 모델 다운로드
model_path = base_path / "yolov8n.pt"
model_url = "https://github.com/ultralytics/assets/releases/download/v8.3.0/yolov8n.pt"

if not model_path.exists():
    print(f"사전 학습된 모델을 다운로드합니다...URL: {model_url}저장 위치: {model_path}")
    try:
        response = requests.get(model_url, stream=True, timeout=30)
        response.raise_for_status()
        with open(model_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print("모델 다운로드 완료.")
    except requests.exceptions.RequestException as e:
        print(f"모델 다운로드 중 오류 발생: {e}")
else:
    print(f"모델이 이미 존재합니다: {model_path}")

모델이 이미 존재합니다: C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-3-Yolov8\yolov8n.pt


In [9]:
from ultralytics import YOLO
from pathlib import Path

# 9. 로컬 모델 파일 로드
model_path = base_path / "yolov8n.pt"

if model_path.exists():
    model = YOLO(model_path)
    print(f"사전 학습된 모델을 성공적으로 로드했습니다: {model_path}")
else:
    print(f"오류: 모델 파일을 찾을 수 없습니다. - {model_path}")
    print("8번 셀에서 모델이 정상적으로 다운로드되었는지 확인하세요.")

사전 학습된 모델을 성공적으로 로드했습니다: C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-3-Yolov8\yolov8n.pt


In [ ]:
# 10. 모델 학습 실행
if 'model' in locals():
    results = model.train(
        data=str(new_yaml_path), # pathlib 객체를 문자열로 변환
        epochs=100,
        patience=30,
        batch=32,
        imgsz=416,
        project=str(base_path), # pathlib 객체를 문자열로 변환
        name='Aquarium_model_results',
        exist_ok=True
    )
    print("모델 학습이 완료되었습니다.")
else:
    print("모델이 정상적으로 로드되지 않아 학습을 시작할 수 없습니다.")

Ultralytics 8.3.214  Python-3.10.18 torch-2.8.0+cpu CPU (Intel Core i5-10400 2.90GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-3-Yolov8\Aquarium_DataSet_Corrected.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=C:\Users\it\Documents\Python-workspace\Ai-ML-DL-workspace-0930\memo\3-3-Yolov8\yolov8n.pt, momentum=0.937, mosaic=1.0,